In [ ]:
!pip install scrapy

In [ ]:
import requests
from scrapy import Selector

response = requests.get("https://cleantechnica.com/")
html_page = response.content
html_sel = Selector(text=html_page)

In [ ]:
title = html_sel.css("h2.zox-s-title2::text").extract()
title

In [ ]:
url = html_sel.css("div.zox-art-title > a::attr(href)").extract()
url

In [ ]:
import requests
from scrapy import Selector

def get_html(url):
  response = requests.get(url)
  content = response.content
  return content

def get_url(sel):
  url = sel.css("div.zox-art-title > a::attr(href)").extract_first()
  return url

def parse_content(sel):
  data = {}
  
  # extract title
  title = sel.css("div.zox-post-head > h1::text").extract_first()
  data["title"] = title

  # extract contents
  # pakai 'or' karena beberapa berita memiliki perbedaan struktur element tags html-nya
  contents = sel.css("div.zox-post-body > p > span::text").extract() or sel.css("div.zox-post-body > p::text").extract() 
  data["contents"] = contents

  # extract date
  date = sel.css("span > time::attr(datetime)").extract_first()
  data["date"] = date

  # extract author
  author = sel.css("div.zox-author-name-wrap > span > a::text").extract_first()
  data["author"] = author

  # extract url
  url = sel.css("div.zox-article-wrap > meta::attr(itemid)").extract_first()
  data["url"] = url

  return data

In [ ]:
html_page = get_html("https://cleantechnica.com/")
html_sel = Selector(text=html_page)

news = html_sel.css("article.zox-art-wrap")

scraps = []
for sel in news:
  url = get_url(sel)
  content_page = get_html(url) # langsung masuk ke masing-masing berita
  content_sel = Selector(text=content_page)
  data = parse_content(content_sel)

  scraps.append(data)

In [ ]:
import pprint

pp = pprint.PrettyPrinter(indent=2)

for scrap in scraps:
  pp.pprint(scrap)

In [ ]:
import json

with open("cleantechnica_scrap.json", "w") as file:
  file.write(json.dumps(scraps))